In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '4g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/09/19 16:00:20 WARN Utils: Your hostname, DESKTOP-3NQ3PQI resolves to a loopback address: 127.0.1.1; using 172.31.183.205 instead (on interface eth0)
22/09/19 16:00:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/19 16:00:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/19 16:00:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [ ]:
sdf_consumer = spark.read.csv('../data/tables/tbl_consumer.csv', sep='|', header=True)
sdf_consumer.show(5, truncate=False)

Convert postcodes to strings with leading zeros

In [ ]:
sdf_consumer = sdf_consumer.withColumn('postcode', F.format_string("%04d", F.col('postcode').cast('int')))

In [ ]:
sdf_consumer.count()

In [ ]:
VIC_sdf = sdf_consumer.where(
    (F.col('state') == 'VIC' ))

NSW_sdf = sdf_consumer.where(
    (F.col('state') == 'NSW' ))

QLD_sdf = sdf_consumer.where(
    (F.col('state') == 'QLD' ))

WA_sdf = sdf_consumer.where(
    (F.col('state') == 'WA' ))

TAS_sdf = sdf_consumer.where(
    (F.col('state') == 'TAS' ))

SA_sdf = sdf_consumer.where(
    (F.col('state') == 'SA' ))

ACT_sdf = sdf_consumer.where(
    (F.col('state') == 'ACT' ))

NT_sdf = sdf_consumer.where(
    (F.col('state') == 'NT' ))

In [ ]:
VIC_sdf.count()
VIC_sdf= VIC_sdf.where(
    ((F.col('postcode') >=  '3000') & (F.col('postcode') <= '3999'))
    | ((F.col('postcode') >=  '8000') & (F.col('postcode') <= '8999'))
)
VIC_sdf.count()

In [ ]:
NT_sdf.count()
NT_sdf= NT_sdf.where(
    (F.col('postcode') >=  '800') & (F.col('postcode') <= '999'))
NT_sdf.count()

In [ ]:
TAS_sdf.count()
TAS_sdf= TAS_sdf.where(
    (F.col('postcode') >=  '7000') & (F.col('postcode') <= '7999'))
TAS_sdf.count()

In [ ]:
WA_sdf.count()
WA_sdf= WA_sdf.where(
    ((F.col('postcode') >=  '6000') & (F.col('postcode') <= '6797'))
    | ((F.col('postcode') >=  '6800') & (F.col('postcode') <= '6999'))
)
WA_sdf.count()

In [ ]:
SA_sdf.count()
SA_sdf= SA_sdf.where(
    (F.col('postcode') >=  '5000') & (F.col('postcode') <= '5999'))
SA_sdf.count()

In [ ]:
QLD_sdf.count()
QLD_sdf= QLD_sdf.where(
    ((F.col('postcode') >=  '4000') & (F.col('postcode') <= '4999'))
    | ((F.col('postcode') >=  '9000') & (F.col('postcode') <= '9999'))
)
QLD_sdf.count()

In [ ]:
NSW_sdf.count()
NSW_sdf= NSW_sdf.where(
    ((F.col('postcode') >=  '1000') & (F.col('postcode') <= '2599'))
    | ((F.col('postcode') >=  '2619') & (F.col('postcode') <= '2899'))
    | ((F.col('postcode') >=  '2921') & (F.col('postcode') <= '2999'))
)
NSW_sdf.count()

In [ ]:
ACT_sdf.count()
ACT_sdf= ACT_sdf.where(
    ((F.col('postcode') >=  '200') & (F.col('postcode') <= '300'))
    | ((F.col('postcode') >=  '2600') & (F.col('postcode') <= '2618'))
    | ((F.col('postcode') >=  '2900') & (F.col('postcode') <= '2920'))
)
ACT_sdf.count()

In [ ]:
cleaned_consumer = VIC_sdf.union(NSW_sdf)
cleaned_consumer = cleaned_consumer.union(QLD_sdf)
cleaned_consumer = cleaned_consumer.union(WA_sdf)
cleaned_consumer = cleaned_consumer.union(TAS_sdf)
cleaned_consumer = cleaned_consumer.union(SA_sdf)
cleaned_consumer = cleaned_consumer.union(ACT_sdf)
cleaned_consumer = cleaned_consumer.union(NT_sdf)

cleaned_consumer.count()

In [ ]:
cleaned_consumer.write.mode('overwrite').parquet('../data/curated/cleaned_consumers.parquet')